# Base firestore

* jump to relevant section after this

In [1]:
import os,importlib
from datetime import datetime
import pandas as pd
from IPython import display
from IPython.display import display, Markdown,Image

import firebase_admin
from firebase_admin import firestore, storage, credentials

" Local includes"
from util import getFileName, getFileIsoDate, dir2df

class Firestore:
    def __init__(self,
                 service_account_file="C:/i/auth/run-pix-092258e3cb1b.json",
                 bucket_name='run-pix.appspot.com'):
        cred = credentials.Certificate(service_account_file)
        try: self.app = firebase_admin.initialize_app(cred)
        except: self.app = firebase_admin.get_app()
        self.fstore=firestore.client()
        self.storage=firebase_admin.storage
        self.bucket=firebase_admin.storage.bucket(name=bucket_name, app=None) 
    def getDoc(self,path):
        return self.fstore.document(path).get().to_dict()
    def updateDoc(self,path,values):
        return self.fstore.document(path).update(values)
    
    def collection(self):
        return self.fstore
    
    def fs2Df(self,collPath):
        def _f(x): # dict --> array of dicts
            _={"id":x.id,'ref':x}
            try: _.update(x.get().to_dict())
            except: pass
            return _
        return pd.DataFrame([_f(_) for _ in self.fstore.collection(collPath).list_documents()])    
    "delete all subnodes"
    def delete_all_docs(self,collection):
        for n in collection.get():
            print(n.__dict__)
            n._reference.delete()
   
runpix=Firestore()

bucket=firebase_admin.storage.bucket(name='run-pix.appspot.com', app=None) 

class Race:
    storage_prefixes='processed uploads thumbs'.split()
    status=[]
    blobs={}
    bibs=[]
    images=[]
    readings=[]
    def __init__(self,id):
        self.fstore=firestore.client()
        self.bucket=firebase_admin.storage.bucket(name='run-pix.appspot.com', app=None)         
        self.id = id
        for k,v in self.fstore.document(f'races/{id}').get().to_dict().items():
          setattr(self,k,v)
    def __repr__(self):
        return f"""<Race:{self.id} bibs:{len(self.bibs)} Images:{len(self.images)} readings:{len(self.readings)}
        status: {self.status}
        blobs:{[len(self.blobs[_].keys()) for _ in self.blobs.keys()]}>
        """
    def getBlobs(self):
        for folder in self.storage_prefixes:
            self.blobs[folder]={b.name:{"id":b.id,"name":b.name,"blob":b,"size":b.size} 
                                    for b in bucket.list_blobs(prefix=f"{folder}/{self.id}")}
        return self.blobs
        # self.blobs_df=pd.DataFrame(
    def getBibs(self):
        df_d=[]
            # .where(u'textAnnotations', u'==', [])
        docs = runpix.fstore.collection(u'races',self.id,'bibs').stream()
        for i,doc in enumerate(docs):
            df_d.append(doc.to_dict())
            # self.bibs=[_.get().to_dict() for _ in self.fstore.collection(f'races/{self.id}/bibs').list_documents()]
        self.bibs=df_d
        return df_d
    def getReadings(self):
        df_d=[]
            # .where(u'textAnnotations', u'==', [])
        docs =  runpix.fstore.collection(u'races',self.id,'readings').stream()
        for i,doc in enumerate(docs):
            df_d.append(doc.to_dict())
            # self.bibs=[_.get().to_dict() for _ in self.fstore.collection(f'races/{self.id}/bibs').list_documents()]
        self.readings=df_d
        return df_d 
    def getimages(self,n=None):
        df_d=[]
            # .where(u'textAnnotations', u'==', [])
        docs = runpix.fstore.collection(u'races',self.id,'images').stream()
        for i,doc in enumerate(docs):
            data_d=doc.to_dict()
            ob=data_d
            # ob={k:v for k,v in data_d.items() if k in 'imagePath texts'}
            # if 'texts' in data_d:
                # {'texts':data_d['texts']})
                # print(f'{doc.id} => {len(data_d["texts"])}')
            # else:
                # df_d.append({'texts':[]})
                # print(f'{doc.id} => non textAnnotations')
            df_d.append(ob)
            if n!=None and i>n:
                break
        self.images=df_d
        return df_d
    def saveBib(self,bibData):
      _bibData=bibData if isinstance (bibData,dict) else bibData.to_dict()
      _bibData['Bib']=str(_bibData['Bib'])
      _bibData['Name']=_bibData['Name'].upper()
      _path=f"races/{self.id}/bibs/{_bibData['Bib']}"
      runpix.fstore.document(_path).set(_bibData)
      print(f"loaded {_bibData['Bib']} {_bibData['Name']}")

    def deleteBlobs(self,folders=['thumbs']):
        for folder in folders:
            for b in race.blobs[folder]:
                try:
                    runpix.bucket.delete_blob(race.blobs[folder][b]['name'])
                    print(race.blobs[folder][b]['name'])
                except:
                    print(f"Error deleting {race.blobs[folder][b]['name']}")


# Load up a race

In [2]:
raceId="mychoice23apr"

In [3]:
%time
race= Race(raceId)
race.getBlobs()
race.getBibs()
race.getimages()
race.getReadings()

# x=[_.get().to_dict() for _ in self.fstore.collection(f'races/{self.id}/bibs').list_documents()]
race

CPU times: total: 0 ns
Wall time: 0 ns


<Race:mychoice23apr bibs:739 Images:1832 readings:2994
        status: ['started', 'stopped']
        blobs:[1809, 1870, 1808]>
        

In [4]:
def searchImage(search):
    return [_['imagePath'] for _ in race.images if search in _['imagePath']]
# def setfireStorage
display(searchImage("50Z"))
img0=searchImage("16.950Z")[0]

['2023-04-09T01:03:59.150Z~5~avinashmane$gmail.com~capture.png',
 '2023-04-09T01:04:37.350Z~5~avinashmane$gmail.com~capture.png',
 '2023-04-09T01:09:21.150Z~5~avinashmane$gmail.com~capture.png',
 '2023-04-09T01:11:24.150Z~5~avinashmane$gmail.com~capture.png',
 '2023-04-09T01:17:09.650Z~5~avinashmane$gmail.com~capture.png',
 '2023-04-09T01:25:03.550Z~5~avinashmane$gmail.com~capture.png',
 '2023-04-09T01:29:40.550Z~5~avinashmane$gmail.com~capture.png',
 '2023-04-09T01:53:29.750Z~VENUE~avinashmane$gmail.com~IMG_20230409_072326.jpg',
 '2023-04-09T01:53:39.950Z~VENUE~avinashmane$gmail.com~IMG_20230409_072338.jpg',
 '2023-04-09T02:31:16.950Z~VENUE~breakingcoconut$gmail.com~IMG_20230409_080116.jpg',
 '2023-04-09T16:05:16.650Z~VENUE~jparagj$gmail.com~1P6A7097.jpg',
 '2023-04-09T16:17:47.450Z~VENUE~jparagj$gmail.com~1P6A7265.jpg',
 '2023-04-09T16:30:07.450Z~VENUE~jparagj$gmail.com~1P6A7419.jpg',
 '2023-04-09T16:40:53.650Z~VENUE~jparagj$gmail.com~1P6A7553.jpg',
 '2023-04-09T17:05:17.950Z~VENUE~j

In [5]:
from IPython.display import Image


# display image

In [176]:
from PIL import Image, ImageDraw, ImageFont
from matplotlib.pyplot import imshow, figure
# from matplotlib import pyplot as plt
import io


# im.show()
class rpImage:
    # img0=f"races/{race.id}/images/{img0}"
        
    def __init__(self,img="races/mychoice23apr/images/2023-04-09T16:04:59.925Z~VENUE~jparagj$gmail.com~1P6A7093.jpg"):
        self.fsPath=img
        folder='uploads'
        self.imgBlobPath="/".join([folder,race.id,img.split("/")[-1]])
        display(self.imgBlobPath)
        self.fs=runpix.getDoc(img)
        
        self.blob=runpix.bucket.blob(self.imgBlobPath)
        self.im = Image.open(io.BytesIO(self.blob.download_as_bytes()))
   
    def mapxy(self,xy): return xy['x'],xy['y']
    def mapBoundingPoly(self,x):
        arr=[]
        for i in [0,2]:
            arr.append(self,mapxy(x['vertices'][i]))
        return arr 
    def testMap(self):
        for _a in self.fs['textAnnotations']:
            print(_a['description'][:10],mapBoundingPoly(_a['boundingPoly']))
            
    def show(self,inline=True):
        if inline:
            figure(figsize = (12,8))
            imshow(self.im)   
        else:
            self.im.show()
    def drawAnnotations(self):
        # use a truetype font
        self.font = ImageFont.truetype("arial.ttf", 40)
        self.draw = ImageDraw.Draw(self.im)            
        # xy=[(x0, y0), (x1, y1)] or [x0, y0, x1, y1]
        # draw.rectangle([100,200,400,500], fill=None, outline=None, width=5)
        for i,_a in enumerate(self.fs['textAnnotations']):
            # print(_a['description'][:10],mapBoundingPoly(_a['boundingPoly']))
            # fnt = ImageFont.truetype("Pillow/Tests/fonts/FreeMono.ttf", 40)
            self.draw.rectangle(mapBoundingPoly(_a['boundingPoly']), fill=None, outline="#ffff0014", width=5)
            for i in range(4):
                # print(f"draw.line([{mapxy(_a['boundingPoly']['vertices'][i]),mapxy(_a['boundingPoly']['vertices'][(i+1)%4])}], fill=None, width=0, joint=None)")
                self.draw.line([mapxy(_a['boundingPoly']['vertices'][i]),mapxy(_a['boundingPoly']['vertices'][(i+1)%4])], 
                               fill="#ffff0014", width=20,joint=None) #(255, 255, 0, 128)
            # draw text, half opacity
            try:
                self.draw.text(mapxy(_a['boundingPoly']['vertices'][0]), _a['description'], 
                               font=self.font, fill="#ff000014")
            except:
                print('err')
                pass



In [179]:
img=rpImage('races/mychoice23apr/images/2023-04-09T16:10:15.695Z~VENUE~jparagj$gmail.com~1P6A7169.jpg')
# img.__dict__

'uploads/mychoice23apr/2023-04-09T16:10:15.695Z~VENUE~jparagj$gmail.com~1P6A7169.jpg'

In [182]:
# img.show()

img.drawAnnotations()
img.show(False)

# Move metadata and Annotations to image metadata

In [218]:
img.blob.metadata=img.fs['metadata']
len(str(img.fs['metadata']))
len(str(img.fs['textAnnotations']))


4637

# test with Vision API

In [232]:
%env GOOGLE_APPLICATION_CREDENTIALS='c:/i/auth/run-pix-092258e3cb1b.json'
import os
# credential_path = r"c:\i\auth\run-pix-092258e3cb1b.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'c:/i/auth/run-pix-092258e3cb1b.json'

env: GOOGLE_APPLICATION_CREDENTIALS='c:/i/auth/run-pix-092258e3cb1b.json'


In [248]:
def detect_crop_hints(path,aspectratios):
    """Detects crop hints in an image."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()
    im = Image.open(io.BytesIO(content))
    draw = ImageDraw.Draw(im)          
    
    image = vision.Image(content=content)

    crop_hints_params = vision.CropHintsParams(aspect_ratios=aspectratios)
    image_context = vision.ImageContext(
        crop_hints_params=crop_hints_params)

    response = client.crop_hints(image=image, image_context=image_context)
    hints = response.crop_hints_annotation.crop_hints

    for n, hint in enumerate(hints):
        print('\nCrop Hint: {}'.format(n))

        vertices = [(vertex.x,vertex.y)
                    for vertex in hint.bounding_poly.vertices]
        draw.rectangle([vertices[0],vertices[2],], fill=None, outline="#ffff0014", width=5)
        print(vertices)

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
        
 

    # imshow(im)
    im.show()
        
    return hints
        
detect_crop_hints(r"C:\temp\a21.jpg",[.66,.5,1])        


Crop Hint: 0
[(1018, 0), (3013, 0), (3013, 3023), (1018, 3023)]

Crop Hint: 1
[(1260, 0), (2772, 0), (2772, 3023), (1260, 3023)]

Crop Hint: 2
[(504, 0), (3528, 0), (3528, 3023), (504, 3023)]


[bounding_poly {
  vertices {
    x: 1018
  }
  vertices {
    x: 3013
  }
  vertices {
    x: 3013
    y: 3023
  }
  vertices {
    x: 1018
    y: 3023
  }
}
confidence: 0.59375
importance_fraction: 0.660000086
, bounding_poly {
  vertices {
    x: 1260
  }
  vertices {
    x: 2772
  }
  vertices {
    x: 2772
    y: 3023
  }
  vertices {
    x: 1260
    y: 3023
  }
}
confidence: 0.59375
importance_fraction: 0.5
, bounding_poly {
  vertices {
    x: 504
  }
  vertices {
    x: 3528
  }
  vertices {
    x: 3528
    y: 3023
  }
  vertices {
    x: 504
    y: 3023
  }
}
confidence: 0.59375
importance_fraction: 1
]

In [214]:
# This snippet has been automatically generated and should be regarded as a
# code template only.
# It will require modifications to work:
# - It may require correct/in-range values for request initialization.
# - It may require specifying regional endpoints when creating the service
#   client as shown in:
#   https://googleapis.dev/python/google-api-core/latest/client_options.html
from google.cloud import vision
# from oauth2client.service_account import ServiceAccountCredentials


ModuleNotFoundError: No module named 'oauth2client'

In [201]:
client = vision.ImageAnnotatorClient(credentials="/i/auth/run-pix-092258e3cb1b.json")

In [204]:
vision.AnnotateImageRequest?

Init signature:
vision.AnnotateImageRequest(
    mapping=None,
    *,
    ignore_unknown_fields=False,
    **kwargs,
)
Docstring:     
Request for performing Google Cloud Vision API tasks over a
user-provided image, with user-requested features, and with
context information.

Attributes:
    image (google.cloud.vision_v1.types.Image):
        The image to be processed.
    features (MutableSequence[google.cloud.vision_v1.types.Feature]):
        Requested features.
    image_context (google.cloud.vision_v1.types.ImageContext):
        Additional context that may accompany the
        image.
File:           c:\sw\py310nb_env\lib\site-packages\google\cloud\vision_v1\types\image_annotator.py
Type:           MessageMeta
Subclasses:     

In [ ]:
# Initialize request argument(s)
request = vision.BatchAnnotateImagesRequest(
)

# Make the request
response = client.batch_annotate_images(request=request)

# Handle the response
print(response)

In [197]:
# import argparse
# import io

# from google.cloud import vision
# from google.cloud.vision import types
!pip install google-cloud-vision

     ---------------------------------------- 0.0/444.3 kB ? eta -:--:--
     ------------------------------------  440.3/444.3 kB 13.4 MB/s eta 0:00:01
     ------------------------------------  440.3/444.3 kB 13.4 MB/s eta 0:00:01
     -------------------------------------- 444.3/444.3 kB 3.9 MB/s eta 0:00:00


# Races 

In [ ]:
races=runpix.fstore.collection('races')
"list all races"

df_races=fs2Df('races')
races.where("Location","==","GT").select("Name Location".split()).

## list all blobs

In [ ]:
df_blob[:3].apply(lambda x: x['name'].split("/").pop(),
                  axis=1)

## other

In [ ]:
"query examples"
{_.id:_.to_dict() for _ in races.where('Location','==','GT').stream()}

In [ ]:
"""
COPY RACE
"""
def copyRace(fromId,toId):
    fromRace=races.document(fromId).get().to_dict()
    print(fromRace)
    return races.document(toId).set(fromRace)

# copyRace('mychoice23apr','mychoice23mar')
# copyRace('mychoice23mar','mychoice23jan')

In [ ]:
def addNewRace(id):
    raceData = {'Date': "YYYY-MM-DD", 
                'Waypoints': ['venue', 'start', 'end'], 
                'Name': 'Default Race Year Month', 
                'Location': 'DC',
                'bibPatterm': r'\d\d\d\d'
               }
    update_time, city_ref = races.document(id).set(raceData)
    print(f'Added document with id {city_ref.id}')
    
addNewRace('mychoice23apr')

# Overall Dashboard

In [ ]:
df_races.style

In [ ]:
def getDir(x):
    return pd.Series(x.split('/')[:-1]+3*[''])[:3]
    
def printBlobSummary(df):
    df['size_mb']=df['size']/10**6
    return pd.concat([df,df.name.apply(getDir)],axis=1)#).str.split("[\-/]",n=2,regex=True,expand=True)

printBlobSummary(df_blob).pivot_table(index=[0,1],values="size_mb",aggfunc=['sum','count']).style

# Images (Storage)

In [ ]:

df_raceImages=Race('mychoice23mar').getRaceImages()
# old df_raceImages=getRaceImages('mychoice23mar')

display(df_raceImages.shape)

In [ ]:
"""
    images without texts
"""
# df_raceImages.query("texts.isna()")


In [ ]:
df_raceImages['texts'].explode().value_counts().reset_index().style

# Storage

In [ ]:
# for i, b in enumerate(blobs):
#     print (i,b.name)
all_blobs_l=df_blob.name.apply(lambda x : x.split("~")[-1])

In [ ]:
"signed URL"
blobs[3].generate_signed_url(pd.Timestamp.today())

In [ ]:

from base64 import b64decode
base64_data = blocs[1].download_as_bytes() #"iVBORw0KGgoAAAANSUhEUgAABL ...  the rest of data "

In [ ]:
base64_data[:100]

In [ ]:
display.Image(b64decode(base64_data))

In [ ]:
# def newname(x):
#     arr=x.split('/')
#     arr[]
# gs://run-pix.appspot.com/thumbs/2023-02-12T01:25:41.084Z^venue^avinashmane$gmail.com^20230212_065538.jpg/2023-02-12T01:25:41.084Z^venue^avinashmane$gmail.com^20230212_065538.jpg
for i,b in enumerate(blobs):
    if 'thumbs/2023' in b.name :#and b.content_type=='image/jpeg':        
        newName="thumbs/mychoice23feb/"+b.name.split('/')[-1]
        print(i,b.name,newName)
        bucket.copy_blob(b,bucket,newName)

In [ ]:

".generate_signed_url() needs date"
new_id="run-pix.appspot.com/thumbs/mychoice23feb/2023-02-12T01:25:41.084Z^venue^avinashmane$gmail.com^20230212_065538.jpg/1678882880166980"
blobs[72].content_type

In [ ]:
bucket.copy_blob()

# Bulk Bib Upload

Data copied from clipboard from Excel:

Columns copied are (case sensitive)
* Bib
* Name
* Status
* Race

In [ ]:
# raceId = 'werun2023'
display(Markdown(f"## Creating Bib list for {raceId}"))
bibs=runpix.fstore.collection(f'races/{raceId}/bibs')

In [ ]:
"Check Race details"
def dictUtf(mydict): 
    return {k: str(v).encode("utf-8") for k,v in mydict.items()}

races.document(raceId).get().to_dict()

##  Copy the data
Data copied from clipboard from Excel:

Columns copied are (case sensitive)
* Bib
* Name
* Status
* Race

In [ ]:
df_startlist=pd.read_clipboard()
display("columns in clipboard:",df_startlist.columns)
if 'BIB Number' in df_startlist:
    df_startlist=df_startlist.rename(columns={'BIB Number': 'Bib', 'fullname': 'Name',  })
    df_startlist['Status']='From sheet'
    df_startlist['Race']='My Choice'
_required_columns='Bib	Name	Status	Race'.split('\t')
_found_columns = [_x for _x in _required_columns if  _x in df_startlist.columns]
display("columns found ",_found_columns)
if len(_found_columns)<len(_required_columns): print("PLEASE WAIT "+('X'*10+' ')*3)


In [ ]:
df_startlist=df_startlist[_required_columns]

In [ ]:
"code"
bibtoDict=lambda x: x.to_dict()
bibtoDict(df_startlist.loc[0,:])  # test

def saveBibtoRace(bibData,bibsCollection):
    _bibData=bibtoDict(bibData)
    _bibData['Bib']=str(_bibData['Bib'])
    bibsCollection.document(_bibData['Bib']
                           ).set(_bibData)
saveBibtoRace(df_startlist.loc[0,:],bibsCollection=bibs,)  # test    
# df_startlist.apply(, axis=1)


In [ ]:
" upload all bibs"
# df_startlist.apply(bibtoDict, axis=1)  #test
df_startlist.apply(lambda x: saveBibtoRace(x,bibs), axis=1) 

In [ ]:
bibs.count().get()
# len(list( bibs.list_documents()))

# Update images to bibs 

Add list of images to the bibs

In [ ]:
df_dir.head()

# Bulk Upload images list_documents

In [ ]:
import yaml
import os
cfg={}
cfg.update(yaml.safe_load(r"""
uploads:
    werun2023:
        #path: D:\We Run 2023\JPEG
        path: D:\We Run 2023\Vaibhav\JPEG
        fileRange: ["","XX"]
        waypoint: venue
        userid: vaibhav
    mychoice23feb:
        path: D:\umesh\D K D  2023 2\Untitled Export
        waypoint: general
        userid: bcoconut
    mychoice23mar:
        path: D:\DKD-Parag\DKD 2023\DCIM\New folder
        waypoint: general
        userid: bcoconut
"""))


raceId='mychoice23mar'
cfg['uploads'][raceId]

In [ ]:
df_dir=dir2df(cfg['uploads'][raceId]['path'])
# df_dir['type root'.split()].value_counts()                 
print(f"{raceId} uploading from '{cfg['uploads'][raceId]['path']}',df_dir:{df_dir.shape}\n",df_dir.root.value_counts())

In [ ]:
"""
Manually rename to retrigger the processing...
- You can change waypoint from venue to general
"""
"* Storage: /uploads/race/time~wpt~user~loc~file    # uploaded images"
i=0
folder=f'uploads/{raceId}/'
import concurrent.futures 
def getNewfileName(file,
                   user=cfg['uploads'][raceId]['userid'],
                   place=cfg['uploads'][raceId]['waypoint']):
    return "~".join([getFileIsoDate(file),place,user,getFileName(file)])

def uploadFile(root,name):
    path=os.path.join(root, name)
    new=folder+getNewfileName(path)
    # blob=bucket.blob(folder+name) #used when renaming to diff filenames
    blob=bucket.blob(new)

    if blob.exists():
        print(stats['files'],'blob exists/rewriting',name,new)
        blob.rewrite(blob)
    else:
        threads.append({new:executor.submit(blob.upload_from_filename,path)})
        print(stats['files'],f'uploading {path} to {new}', )
        # blob.upload_from_filename(path)
        

stats={"files":0,'upl':0}
prefix=f'processed/{raceId}'

all_blobs_l=[blob.name.split("~")[-1] for blob in bucket.list_blobs(prefix=prefix)]
executor=concurrent.futures.ThreadPoolExecutor(max_workers=5)
threads=[]

for lot in cfg['uploads'].keys():
    _dir=cfg['uploads'][lot]['path'].lower()
    print("raceId",lot,_dir,f"{len(all_blobs_l)} in {prefix}  ====================")
    # for root, dirs, files in os.walk(cfg['uploads'][lot]['path'], topdown=False):
    for i,d in df_dir.query(f"(type=='file') and (root.str.lower() == @_dir)"
                             ).iterrows():
        stats['files']+=1
        if not d.path in all_blobs_l:
            uploadFile(d.root,d.path)
            stats['upl']+=1
        else:
            print(prefix,d.path ,"found")
            pass
        # if (stats['files']>20): raise  #stopper
    print(f"{lot}: {stats['files']} files")
print(stats)

In [ ]:
"""
    find object from gcs
    
"""
display(Markdown("## rename blob"))
# bucket.blob('uploads/werun2023/_L3A3192.jpg').rewrite
# blob.rewrite?
bname='2023-03-13T19:23:14.739819~general~vaibhav~_L3A2997.jpg'
def renameBlob(bname):
    bucket.rename_blob(bucket.blob(bname),folder+bname)
# renameBlob(bname) 
"use 2: existence check"
bucket.blob('error in getDownloadURL thumbs/werun2023/2023-03-13T19:23:42.882584~general~vaibhav~_L3A3007.jpg').exists()

## check storage file (which folder?)

In [ ]:
# check existance
# _path=r"processed/mychoice23feb/2023-02-12T01:28:29.364Z^venue^avinashmane$gmail.com^20230212_065828.jpg"
_path="processed/werun2023/2023-03-13T19:25:41.041091~general~vaibhav~_L3A3047.jpg"
[_,_raceId,_name]=_path.split("/")

print(_,_raceId,_name,)
for r in ['default',_raceId]:
    for t in 'processed thumbs uploads'.split():
        newName="/".join([t,r,_name])
        _blob=bucket.blob(newName)
        print(newName,_blob.exists() )


## check all images in the race

In [ ]:
df_images=fs2Df(f'races/{raceId}/images')
df_images.shape

In [ ]:
# df_images['imagePath'].value_counts()
def checkBlobs(x,types=['processed','thumbs']):
    return [bucket.blob(f'{typ}/{raceId}/{x}').exists() 
                                        for typ in types]
def moveBlob(x,typ='thumbs'): #'processed',
    correctName=f'{typ}/{raceId}/{x}'
    if not bucket.blob(correctName).exists() :
        defaultThumb=bucket.blob(f'{typ}/default/{x}')
        
        if defaultThumb.exists():
            bucket.rename_blob(defaultThumb,f'{typ}/{raceId}/{x}')
            print(f"{correctName} not exists, moving from default")
        else:
            procBlob=bucket.blob(f'uploads/{raceId}/{x}')
            if procBlob.exists():
                print(f"{procBlob.name} rename the blob to retrigger functions")
                procBlob.rewrite(procBlob)
                
            else:
                procBlob.upload_from_filename(path4file+x.split("~")[-1])
                print(f"need to upload {x}")
    else:
        print(f"{correctName} exists")

path4file=r"D:\We Run 2023\JPEG\\"
display(raceId, path4file)
df_images['imagePath'].apply(lambda x: moveBlob(x))

In [ ]:
def deleteFSimages4missingBlobs(x,types=['processed','thumbs']):
    chk = checkBlobs(x.imagePath)
    if not any(chk):
        print(chk,x,x.imagePath)
        x.ref.delete()
df_images.apply(deleteFSimages4missingBlobs ,axis=1)

In [ ]:
df_blob.query("name.str.contains('G017')")

# move from mychoice23APR to Ahimsa


In [ ]:
mychoice23apr

In [ ]:
# allDocs=
""" COPY FIREBASE DATA """
for x in mychoice23apr.fstore.collection('races/mychoice23apr/images').stream():
    if '2023-04-02' in x.id:
        newDict = x.to_dict()
        newDict['metadata']['imagePath']=newDict['metadata']['imagePath'].replace('mychoice23apr','ahimsarun2023')
        # {k:(v.replace() 
        #            if isinstance(v,str) else v)
        #            for (k,v) in .items()}
        mychoice23apr.fstore.document(f'races/ahimsarun2023/images/{x.id}').set(newDict)
        mychoice23apr.fstore.document(f'races/mychoice23apr/images/{x.id}').delete()
        print (x.id)
    else:
        print ('skipping', x.id)

In [ ]:
" move blobs from on race to another "
def copy_blob(name):
    # name="processed/mychoice23apr/2023-03-18T02:39:47.304Z~venue~avinashmane$gmail.com~capture.jpg"
    new_name=name.replace("mychoice23apr","ahimsarun2023")
    print("renaming",name,new_name)
    blob=mychoice23apr.storage.blob(name)
    if blob.exists():
        runpix.bucket.copy_blob(blob,runpix.bucket,new_name=new_name)
        pass
    
for r in "mychoice23apr".split():
    for folder in "processed".split():
        prefix=f"{folder}/{r}/2023-04-02"
        for i,blob in enumerate(mychoice23apr.storage.list_blobs(prefix=prefix)):
            # print(blob.name)
            copy_blob(blob.name)
            blob.delete()
            if i>100: break

In [ ]:
blob.delete()
Image(blob.download_as_bytes())

In [ ]:
copy_blob('processed/mychoice23apr/2023-04-02T00:43:14.291Z~venue~avinashmane$gmail.com~capture.jpg')

# Other

In [ ]:
bucket.blob('processed/werun2023/2023-03-13T19:25:41.041091~general~vaibhav~_L3A3047.jpg').public_url

#Files to be deleted

raceId mychoice23feb d:\umesh\d k d  2023 2\untitled export 3 in processed/mychoice23feb
1 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5534.jpg to uploads/werun2023/2023-03-16T13:57:47.954854~general~bcoconut~1P6A5534.jpg
2 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5535.jpg to uploads/werun2023/2023-03-16T13:57:48.517336~general~bcoconut~1P6A5535.jpg
3 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5536.jpg to uploads/werun2023/2023-03-16T13:57:48.861077~general~bcoconut~1P6A5536.jpg
4 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5537.jpg to uploads/werun2023/2023-03-16T13:57:49.329810~general~bcoconut~1P6A5537.jpg
5 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5538.jpg to uploads/werun2023/2023-03-16T13:57:49.829797~general~bcoconut~1P6A5538.jpg
6 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5539.jpg to uploads/werun2023/2023-03-16T13:57:50.673521~general~bcoconut~1P6A5539.jpg
7 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5540.jpg to uploads/werun2023/2023-03-16T13:57:51.251623~general~bcoconut~1P6A5540.jpg
8 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5541.jpg to uploads/werun2023/2023-03-16T13:57:52.095346~general~bcoconut~1P6A5541.jpg
9 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5542.jpg to uploads/werun2023/2023-03-16T13:57:52.798449~general~bcoconut~1P6A5542.jpg
10 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5543.jpg to uploads/werun2023/2023-03-16T13:57:53.782791~general~bcoconut~1P6A5543.jpg
11 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5544.jpg to uploads/werun2023/2023-03-16T13:57:54.360898~general~bcoconut~1P6A5544.jpg
12 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5545.jpg to uploads/werun2023/2023-03-16T13:57:54.907755~general~bcoconut~1P6A5545.jpg
13 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5546.jpg to uploads/werun2023/2023-03-16T13:57:55.392115~general~bcoconut~1P6A5546.jpg
14 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5547.jpg to uploads/werun2023/2023-03-16T13:57:55.892099~general~bcoconut~1P6A5547.jpg
15 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5548.jpg to uploads/werun2023/2023-03-16T13:57:56.204588~general~bcoconut~1P6A5548.jpg
16 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5549.jpg to uploads/werun2023/2023-03-16T13:57:56.626450~general~bcoconut~1P6A5549.jpg
17 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5550.jpg to uploads/werun2023/2023-03-16T13:57:57.032689~general~bcoconut~1P6A5550.jpg
18 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5551.jpg to uploads/werun2023/2023-03-16T13:57:57.501423~general~bcoconut~1P6A5551.jpg
19 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5552.jpg to uploads/werun2023/2023-03-16T13:57:57.938908~general~bcoconut~1P6A5552.jpg
20 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5553.jpg to uploads/werun2023/2023-03-16T13:57:58.438893~general~bcoconut~1P6A5553.jpg
21 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5554.jpg to uploads/werun2023/2023-03-16T13:57:59.110745~general~bcoconut~1P6A5554.jpg
22 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5555.jpg to uploads/werun2023/2023-03-16T13:57:59.876345~general~bcoconut~1P6A5555.jpg
23 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5556.jpg to uploads/werun2023/2023-03-16T13:58:00.516953~general~bcoconut~1P6A5556.jpg
24 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5557.jpg to uploads/werun2023/2023-03-16T13:58:00.923188~general~bcoconut~1P6A5557.jpg
25 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5558.jpg to uploads/werun2023/2023-03-16T13:58:01.376301~general~bcoconut~1P6A5558.jpg
26 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5559.jpg to uploads/werun2023/2023-03-16T13:58:01.907535~general~bcoconut~1P6A5559.jpg
27 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5560.jpg to uploads/werun2023/2023-03-16T13:58:02.610633~general~bcoconut~1P6A5560.jpg
28 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5561.jpg to uploads/werun2023/2023-03-16T13:58:03.048121~general~bcoconut~1P6A5561.jpg
29 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5562.jpg to uploads/werun2023/2023-03-16T13:58:03.532478~general~bcoconut~1P6A5562.jpg
30 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5563.jpg to uploads/werun2023/2023-03-16T13:58:03.969964~general~bcoconut~1P6A5563.jpg
31 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5564.jpg to uploads/werun2023/2023-03-16T13:58:04.469950~general~bcoconut~1P6A5564.jpg
32 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5565.jpg to uploads/werun2023/2023-03-16T13:58:05.126179~general~bcoconut~1P6A5565.jpg
33 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5566.jpg to uploads/werun2023/2023-03-16T13:58:05.501164~general~bcoconut~1P6A5566.jpg
34 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5567.jpg to uploads/werun2023/2023-03-16T13:58:05.923026~general~bcoconut~1P6A5567.jpg
35 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5568.jpg to uploads/werun2023/2023-03-16T13:58:06.313642~general~bcoconut~1P6A5568.jpg
36 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5569.jpg to uploads/werun2023/2023-03-16T13:58:06.766748~general~bcoconut~1P6A5569.jpg
37 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5570.jpg to uploads/werun2023/2023-03-16T13:58:07.094863~general~bcoconut~1P6A5570.jpg
38 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5571.jpg to uploads/werun2023/2023-03-16T13:58:07.594847~general~bcoconut~1P6A5571.jpg
39 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5572.jpg to uploads/werun2023/2023-03-16T13:58:07.938586~general~bcoconut~1P6A5572.jpg
40 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5573.jpg to uploads/werun2023/2023-03-16T13:58:08.282328~general~bcoconut~1P6A5573.jpg
41 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5575.jpg to uploads/werun2023/2023-03-16T13:58:08.735436~general~bcoconut~1P6A5575.jpg
42 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5576.jpg to uploads/werun2023/2023-03-16T13:58:09.141680~general~bcoconut~1P6A5576.jpg
43 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5577.jpg to uploads/werun2023/2023-03-16T13:58:09.672905~general~bcoconut~1P6A5577.jpg
44 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5579.jpg to uploads/werun2023/2023-03-16T13:58:10.047894~general~bcoconut~1P6A5579.jpg
45 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5580.jpg to uploads/werun2023/2023-03-16T13:58:10.376008~general~bcoconut~1P6A5580.jpg
46 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5581.jpg to uploads/werun2023/2023-03-16T13:58:10.704123~general~bcoconut~1P6A5581.jpg
47 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5582.jpg to uploads/werun2023/2023-03-16T13:58:11.110360~general~bcoconut~1P6A5582.jpg
48 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5583.jpg to uploads/werun2023/2023-03-16T13:58:11.454099~general~bcoconut~1P6A5583.jpg
49 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5585.jpg to uploads/werun2023/2023-03-16T13:58:11.922834~general~bcoconut~1P6A5585.jpg
50 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5586.jpg to uploads/werun2023/2023-03-16T13:58:12.297822~general~bcoconut~1P6A5586.jpg
51 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5587.jpg to uploads/werun2023/2023-03-16T13:58:12.657186~general~bcoconut~1P6A5587.jpg
52 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5588.jpg to uploads/werun2023/2023-03-16T13:58:12.969674~general~bcoconut~1P6A5588.jpg
53 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5589.jpg to uploads/werun2023/2023-03-16T13:58:13.438409~general~bcoconut~1P6A5589.jpg
54 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5590.jpg to uploads/werun2023/2023-03-16T13:58:13.735275~general~bcoconut~1P6A5590.jpg
55 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5591.jpg to uploads/werun2023/2023-03-16T13:58:14.079020~general~bcoconut~1P6A5591.jpg
56 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5592.jpg to uploads/werun2023/2023-03-16T13:58:14.407129~general~bcoconut~1P6A5592.jpg
57 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5594.jpg to uploads/werun2023/2023-03-16T13:58:14.703994~general~bcoconut~1P6A5594.jpg
58 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5595.jpg to uploads/werun2023/2023-03-16T13:58:15.000858~general~bcoconut~1P6A5595.jpg
59 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5596.jpg to uploads/werun2023/2023-03-16T13:58:15.391475~general~bcoconut~1P6A5596.jpg
60 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5597.jpg to uploads/werun2023/2023-03-16T13:58:15.813334~general~bcoconut~1P6A5597.jpg
61 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5598.jpg to uploads/werun2023/2023-03-16T13:58:16.360190~general~bcoconut~1P6A5598.jpg
62 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5599.jpg to uploads/werun2023/2023-03-16T13:58:16.703928~general~bcoconut~1P6A5599.jpg
63 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5600.jpg to uploads/werun2023/2023-03-16T13:58:16.953921~general~bcoconut~1P6A5600.jpg
64 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5601.jpg to uploads/werun2023/2023-03-16T13:58:17.250787~general~bcoconut~1P6A5601.jpg
65 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5602.jpg to uploads/werun2023/2023-03-16T13:58:17.563277~general~bcoconut~1P6A5602.jpg
66 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5603.jpg to uploads/werun2023/2023-03-16T13:58:17.844517~general~bcoconut~1P6A5603.jpg
67 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5604.jpg to uploads/werun2023/2023-03-16T13:58:18.219508~general~bcoconut~1P6A5604.jpg
68 uploading D:\umesh\D K D  2023 2\Untitled Export\1P6A5605.jpg to uploads/werun2023/2023-03-16T13:58:18.516370~general~bcoconut~1P6A5605.jpg